In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 64
p1 = .5

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        
        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        
        self.dropout1 = nn.Dropout(p=p1)
        
        
    def forward(self, input):
        # encoder
        x = F.relu(self.batchnorm2(self.conv1(input)))
        x = self.dropout1(x)
        x = F.relu(self.batchnorm3(self.conv2(x)))
        x = self.dropout1(x)
        
        # decoder
        x = F.relu(self.batchnorm2(self.convtrans1(x)))
        x = self.dropout1(x)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.relu(self.batchnorm1(self.convtrans2(x)))
        x = self.dropout1(x)
        x = self.convtrans3(F.interpolate(x, scale_factor=scale_factor))

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]             640
       BatchNorm2d-2           [-1, 64, 16, 16]             128
           Dropout-3           [-1, 64, 16, 16]               0
            Conv2d-4            [-1, 128, 8, 8]          73,856
       BatchNorm2d-5            [-1, 128, 8, 8]             256
           Dropout-6            [-1, 128, 8, 8]               0
   ConvTranspose2d-7             [-1, 64, 8, 8]          73,792
       BatchNorm2d-8             [-1, 64, 8, 8]             128
           Dropout-9             [-1, 64, 8, 8]               0
  ConvTranspose2d-10           [-1, 32, 16, 16]          18,464
      BatchNorm2d-11           [-1, 32, 16, 16]              64
          Dropout-12           [-1, 32, 16, 16]               0
  ConvTranspose2d-13            [-1, 2, 32, 32]             578
Total params: 167,906
Trainable params:

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00272823 (0.03598032), PSNR 25.64118958 (22.58135067), SSIM 0.73614872 (0.53402521)
Finished training epoch 0
Validate: MSE 0.00337163 (0.00280362), PSNR 24.72160149 (25.59415129), SSIM 0.70421159 (0.77532036)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00311093 (0.00279270), PSNR 25.07109833 (25.56390437), SSIM 0.73201728 (0.76332225)
Finished training epoch 1
Validate: MSE 0.00331748 (0.00278756), PSNR 24.79192352 (25.61993373), SSIM 0.70434582 (0.77536298)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00227066 (0.00277848), PSNR 26.43847656 (25.58544274), SSIM 0.78215110 (0.76722556)
Finished training epoch 2
Validate: MSE 0.00338392 (0.00280684), PSNR 24.70579720 (25.58989541), SSIM 0.70427394 (0.77540022)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00371086 (0.00276981), PSNR 24.30525398 (25.59757915), SSIM 0.74573892 (0.76617538)
Finished training epoch 3
Validate: MSE 0.00335872 (0.0

Epoch: 30, MSE 0.00246575 (0.00254786), PSNR 26.08050919 (25.96191880), SSIM 0.76066971 (0.75752910)
Finished training epoch 30
Validate: MSE 0.00315623 (0.00309122), PSNR 25.00831604 (25.21966250), SSIM 0.70649934 (0.77608394)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00234610 (0.00254181), PSNR 26.29654312 (25.97189748), SSIM 0.75231373 (0.75726775)
Finished training epoch 31
Validate: MSE 0.00306710 (0.00276916), PSNR 25.13272667 (25.65920904), SSIM 0.70676649 (0.77652956)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00192091 (0.00254517), PSNR 27.16493988 (25.96533007), SSIM 0.79286152 (0.75739021)
Finished training epoch 32
Validate: MSE 0.00309590 (0.00291557), PSNR 25.09212494 (25.45689518), SSIM 0.70714015 (0.77656021)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00218712 (0.00253505), PSNR 26.60127068 (25.97962557), SSIM 0.77462900 (0.75730559)
Finished training epoch 33
Validate: MSE 0.00306375 (0.00279398), PSNR 

Epoch: 60, MSE 0.00304286 (0.00251139), PSNR 25.16718483 (26.02270819), SSIM 0.73553103 (0.75674880)
Finished training epoch 60
Validate: MSE 0.00335004 (0.00327427), PSNR 24.74949455 (24.97623371), SSIM 0.70620275 (0.77580304)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00227534 (0.00250302), PSNR 26.42953873 (26.03737450), SSIM 0.75999975 (0.75652618)
Finished training epoch 61
Validate: MSE 0.00308803 (0.00292115), PSNR 25.10318375 (25.44802535), SSIM 0.70668977 (0.77614960)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00237628 (0.00250485), PSNR 26.24102592 (26.03547783), SSIM 0.73992658 (0.75678695)
Finished training epoch 62
Validate: MSE 0.00319027 (0.00293433), PSNR 24.96172714 (25.42319362), SSIM 0.70581150 (0.77617849)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00209189 (0.00250466), PSNR 26.79460335 (26.03190301), SSIM 0.76762503 (0.75675943)
Finished training epoch 63
Validate: MSE 0.00313502 (0.00302312), PSNR 

Epoch: 90, MSE 0.00241687 (0.00249180), PSNR 26.16746712 (26.05586419), SSIM 0.76003838 (0.75636578)
Finished training epoch 90
Validate: MSE 0.00377247 (0.00386086), PSNR 24.23373604 (24.27783780), SSIM 0.70505333 (0.77439101)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00289021 (0.00249768), PSNR 25.39070892 (26.04624814), SSIM 0.74424088 (0.75639405)
Finished training epoch 91
Validate: MSE 0.00330488 (0.00315482), PSNR 24.80843544 (25.12805631), SSIM 0.70570213 (0.77581162)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00218470 (0.00249580), PSNR 26.60608482 (26.05167967), SSIM 0.75753796 (0.75637030)
Finished training epoch 92
Validate: MSE 0.00347241 (0.00356346), PSNR 24.59368706 (24.62297090), SSIM 0.70572633 (0.77529239)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00264445 (0.00249300), PSNR 25.77665329 (26.05512064), SSIM 0.74180150 (0.75639498)
Finished training epoch 93
Validate: MSE 0.00325938 (0.00322634), PSNR 

Epoch: 120, MSE 0.00225905 (0.00248239), PSNR 26.46073532 (26.07524410), SSIM 0.76905477 (0.75594581)
Finished training epoch 120
Validate: MSE 0.00317484 (0.00299958), PSNR 24.98278427 (25.33723608), SSIM 0.70544106 (0.77555543)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00264335 (0.00248406), PSNR 25.77845764 (26.07458859), SSIM 0.76739758 (0.75633527)
Finished training epoch 121
Validate: MSE 0.00311086 (0.00301855), PSNR 25.07119370 (25.31151008), SSIM 0.70566189 (0.77558068)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00259972 (0.00248552), PSNR 25.85072708 (26.06929643), SSIM 0.74802077 (0.75604037)
Finished training epoch 122
Validate: MSE 0.00321202 (0.00308548), PSNR 24.93221474 (25.22327199), SSIM 0.70569462 (0.77558988)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00237760 (0.00248085), PSNR 26.23861694 (26.07482314), SSIM 0.75227898 (0.75589569)
Finished training epoch 123
Validate: MSE 0.00361349 (0.00363

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00307443 (0.00290137), PSNR 25.12235451 (25.47456000), SSIM 0.70587504 (0.77594810)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()